In [9]:

from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [10]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
# pc.create_index(
#     name = "rag", dimension = 1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region ="us-east-1")
# )

In [11]:
import json
data = json.load(open('prof_reviews.json'))
data['reviews']

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Dr. Johnson is very knowledgeable and her lectures are clear. However, her exams are tough.'},
 {'professor': 'Prof. Michael Smith',
  'subject': 'Physics',
  'stars': 5,
  'review': 'Prof. Smith is excellent! His explanations are simple and easy to understand.'},
 {'professor': 'Dr. Sarah Lee',
  'subject': 'Chemistry',
  'stars': 3,
  'review': 'Dr. Lee knows her stuff, but sometimes the lectures are too fast-paced.'},
 {'professor': 'Prof. David Brown',
  'subject': 'History',
  'stars': 4,
  'review': 'Prof. Brown is engaging and makes history interesting.'},
 {'professor': 'Dr. Laura Davis',
  'subject': 'Biology',
  'stars': 2,
  'review': "Dr. Davis is a bit disorganized, and it's hard to follow her lectures."},
 {'professor': 'Prof. Robert Miller',
  'subject': 'English',
  'stars': 5,
  'review': 'Prof. Miller is fantastic! He really cares about his students and their progress.'},
 {'pro

In [12]:
processed_data=[]
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input = review['review'],
        model = 'text-embedding-3-small',
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata":{
            "review":review["review"],
            "subject":review["subject"],
            "stars": review["stars"]
        }
    })

In [16]:
processed_data[0]

{'values': [-0.006066599,
  -0.019134887,
  0.036301162,
  0.0041570473,
  0.01746813,
  0.023964543,
  0.003632085,
  0.033597607,
  -0.0037042673,
  0.026143137,
  0.004235792,
  -0.009777429,
  -0.03721985,
  0.012736904,
  -0.025106335,
  0.031077785,
  -0.02273088,
  -0.02622188,
  0.017664991,
  0.0442281,
  0.04853279,
  -0.011824782,
  0.042626962,
  -0.059373267,
  -0.026825588,
  -0.027586784,
  0.027875513,
  0.014948309,
  0.04504179,
  0.003354839,
  0.08168418,
  0.002140863,
  0.013215933,
  0.0017389386,
  -0.042784452,
  0.036931116,
  0.026681224,
  0.035749953,
  0.026195632,
  0.0009030997,
  -0.026773091,
  0.020591658,
  -0.043991864,
  -0.025591927,
  0.036117427,
  -0.0088849915,
  -0.0020965694,
  -0.027455544,
  0.029266665,
  0.042101998,
  -0.034280058,
  0.03690487,
  0.043125678,
  -0.011975709,
  -0.027271807,
  -0.03900472,
  0.014882689,
  0.025801912,
  0.012973138,
  -0.021654706,
  0.077799454,
  -0.035959937,
  0.0005594133,
  -0.028767949,
  -0.027

In [13]:
index = pc.Index('cowlitea')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [18]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}